## 0.- Install geemap
I recommend you to check this link

https://geemap.org/#introduction

In [ ]:
!pip install geemap

## 1.- Import libraries


In [1]:
import ee, geemap, os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

## 2.- Change main folder

In [2]:
os.chdir("/content/drive/MyDrive/PRONACES/python")

## 3.- Initialize geemap

In [10]:
Map = geemap.Map()
Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=OndqQgsweUKS-Tr-rqZ3Wk6UFa8dGy6hKmV2cOuYtjc&tc=TR9eklJ5RUXBUXjPpBXpXKv8hbEIWRXw-fn4MSOqqa8&cc=tvUGIGpYiIDNzUBJEh73KCWrNcuVIno6Qp4QFn6SI_o

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJqmaa_Zf9_E0_gb9Rf8q8TT697PuYioGkEUgT9xRENZ_H6AksFKGWw

Successfully saved authorization token.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## 4.- Define your region of interest

In [11]:
AGEBS = ee.FeatureCollection("users/FIC/AGEBS");

# Draw any shapes on the map using the Drawing tools before executing this code block
feature = Map.draw_last_feature

if feature is None:
  print("is none")
  geom = ee.Geometry.Polygon(
    [
      [
        [-105.42652805609131,17.9681051090771],
        [-102.44373997015381,17.9681051090771],
        [-102.44373997015381,20.277480725296726],
        [-105.42652805609131,20.277480725296726],
        [-105.42652805609131,17.9681051090771]
      ]
    ]
  )
  feature = ee.Feature(geom, {})

roi = feature.geometry()

is none


# 5.- Export data

In [30]:
BeginDate = '2018-07-10';
EndDate = '2022-09-13';

def datesFunction(d):
  return ee.Date(d).format(None, 'America/Mexico_City')

def roiClip(img):
  return img.clip(roi)

def ExportData(geesrc, column, name):
  collection = ee.ImageCollection(geesrc).select(column).filterDate(BeginDate, EndDate).filterBounds(AGEBS).map(roiClip)
  print(f'La cantidad de imagenes en {name} es', collection.size().getInfo())

  dates = ee.List(collection.aggregate_array("system:time_start")).map(datesFunction);
  list_dates = dates.getInfo()

  count     = collection.size().getInfo()
  collection_list = collection.toList(count)

  for i in range(0, count):

    print('Estamos en el proceso ', i)
    coll_img       = ee.Image(collection_list.get(i))

    #date with month
    date = list_dates[i]
    print(date)

    # Statistical zones of gauge stations  
    geemap.zonal_statistics(
      coll_img, 
      AGEBS, 
      f"data/{name}/{name}_{date}.csv", 
      statistics_type='MEAN', 
      decimal_places=2,
      scale= 1000)  

In [31]:
collections = [
  ['COPERNICUS/S5P/NRTI/L3_O3','O3_column_number_density'],
  ['COPERNICUS/S5P/NRTI/L3_NO2','NO2_column_number_density'],
  ['COPERNICUS/S5P/NRTI/L3_HCHO','tropospheric_HCHO_column_number_density'],
  ['COPERNICUS/S5P/NRTI/L3_CO','CO_column_number_density'],
  ['COPERNICUS/S5P/NRTI/L3_SO2','SO2_column_number_density'],
]

for col in collections:
  quimico = col[0].split("_")[1]
  ExportData(col[0], col[1], quimico)

La cantidad de imagenes en O3 es 2520
Estamos en el proceso  0
2018-07-18T15:02:21
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/PRONACES/python/data/O3/O3_2018-07-18T15:02:21.csv
Estamos en el proceso  1
2018-07-19T14:43:22
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/PRONACES/python/data/O3/O3_2018-07-19T14:43:22.csv
Estamos en el proceso  2
2018-07-20T14:23:22
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/PRONACES/python/data/O3/O3_2018-07-20T14:23:22.csv
Estamos en el proceso  3
2018-07-20T14:28:23
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/PRONACES/python/data/O3/O3_2018-07-20T14:28:23.csv
Estamos en el proceso  4
2018-07-21T14:08:22
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/PRONACES/python/data/O3

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/PRONACES/python/data/O3/O3_2018-10-04T15:42:57.csv
Estamos en el proceso  115
2018-10-05T15:22:58
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/PRONACES/python/data/O3/O3_2018-10-05T15:22:58.csv
Estamos en el proceso  116
2018-10-06T15:02:58
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/PRONACES/python/data/O3/O3_2018-10-06T15:02:58.csv
Estamos en el proceso  117
2018-10-07T14:42:57
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/PRONACES/python/data/O3/O3_2018-10-07T14:42:57.csv
Estamos en el proceso  118
2018-10-07T14:47:58
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/PRONACES/python/data/O3/O3_2018-10-07T14:47:58.csv
Estamos en el proceso  119
2018-10-08T14:22:58
Computing statistics ...


KeyboardInterrupt: ignored

'O3'